In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd drive/My\ Drive/PRML-2020/Data\ Contest

[Errno 2] No such file or directory: 'drive/My Drive/PRML-2020/Data Contest'
/content/drive/My Drive/PRML-2020/Data Contest


In [12]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering

# Bikers information

In [19]:
bikers_info = pd.read_csv('./dataset/bikers.csv').replace(to_replace='None', value=0)
biker_ids = list(bikers_info['biker_id'])
bikers_info.index = bikers_info['biker_id']
bikers_info.drop('biker_id', axis=1, inplace=True)
display(bikers_info)

,language_id,location_id,bornIn,gender,member_since,area,time_zone
biker_id,,,,,,,
DB97468391,id,ID,1993,male,02-10-2012,Medan Indonesia,480.0
DF37982273,id,ID,1992,male,29-09-2012,Medan Indonesia,420.0
IC3183725,en,US,1975,male,06-10-2012,Stratford Ontario,-240.0
BI72223848,en,US,1991,female,04-11-2012,Tehran Iran,210.0
DE29017717,id,ID,1995,female,10-09-2012,NaN,420.0
...,...,...,...,...,...,...,...
EA64394129,en,US,1966,female,12-06-2012,NaN,-240.0
EE6388078,id,ID,1997,male,25-10-2012,Medan Indonesia,420.0
CF48659919,id,ID,1995,male,28-10-2012,Jogjakarta Indonesia,420.0


In [20]:
print("No of uniques locations : {0}".format(len(bikers_info['location_id'].unique())))
print("No of uniques areas : {0}".format(len(bikers_info['area'].unique())))
print("No of uniques gender : {0}".format(len(bikers_info['gender'].unique())))

No of uniques locations : 56
No of uniques areas : 2805
No of uniques gender : 3


In [21]:
format = '%d-%m-%Y'
join_date = []
for date in bikers_info['member_since']:
    try:
        dt = datetime.strptime(date, format)
        dt_int = 10000*dt.year + 100*dt.month + dt.day
        join_date.append(dt_int)
    except:
        join_date.append(0)

In [22]:
format = '%Y'
born_date = []
for date in bikers_info['bornIn']:
    try:
        dt = datetime.strptime(date, format)
        dt_int = dt.year
        born_date.append(dt_int)
    except:
        born_date.append(0)

In [23]:
bikers_info['member_since'] = join_date
bikers_info.drop(['language_id'], axis=1, inplace=True)
bikers_info_encoded = pd.get_dummies(bikers_info, columns=['location_id', 'gender', 'area'])
bikers_info_encoded['bornIn'] = born_date
display(bikers_info_encoded)

,bornIn,member_since,time_zone,location_id_AL,location_id_AR,location_id_AZ,location_id_BA,location_id_BG,location_id_BR,location_id_CA,location_id_CN,location_id_CZ,location_id_DE,location_id_DK,location_id_EE,location_id_EO,location_id_ES,location_id_FI,location_id_FR,location_id_GB,location_id_GE,location_id_GR,location_id_HK,location_id_HR,location_id_HU,location_id_ID,location_id_IL,location_id_IN,location_id_IR,location_id_IT,location_id_JP,location_id_KH,location_id_KR,location_id_LA,location_id_LT,location_id_LV,location_id_MK,location_id_MN,location_id_MX,location_id_MY,...,area_Woodland CA,area_Woodland Hills CA,area_Woodland Hills California,area_Woodlands Singapore,area_Woodside California,area_Woodstock ON,area_Woodstock Ontario,area_Worcester Q4,area_Wuhan China,area_Xiamen Shi Fujian China,area_Yangon Burma,area_Yantai 25,area_Yazd Iran,area_Yekaterinburg 71,area_Yerevan 11,area_Yerevan Armenia,area_Yio Chu Kang Singapore,area_Yogyakarta,area_Yogyakarta 10,area_Yokohama 19,area_Yokosuka-shi Kanagawa Japan,area_Yola Nigeria,area_Yonkers New York,area_Yorba Linda CA,area_York ON,area_Zabol,area_Zagazig 14,area_Zagreb Croatia,area_Zamboanga G7,area_Zamboanga City,area_Zanjan Iran,area_Zaragoza Spain,area_Zaria Nigeria,area_Zarqa' Az Zarqa Jordan,area_Zhengzhou,area_Zhonghe Heilongjiang China,area_Zonhoven,area_Zurich 25,area_`Arab Jizan Saudi Arabia,area_undefined undefined
biker_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DB97468391,1993,20121002,480.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DF37982273,1992,20120929,420.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
IC3183725,1975,20121006,-240.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BI72223848,1991,20121104,210.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DE29017717,1995,20120910,420.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EA64394129,1966,20120612,-240.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
EE6388078,1997,20121025,420.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CF48659919,1995,20121028,420.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
bikers_info_encoded.to_csv('./Required data/biker_info.csv')
bikers_dict = dict(zip(biker_ids, list(range(len(biker_ids)))))
pkl.dump(bikers_dict, open('./Required data/bikers_list.pkl', 'wb'))

# Biker network info (not useful anymore)


In [ ]:
bikers_netw_info = pd.read_csv('./dataset/bikers_network.csv')
new_indx = bikers_netw_info['biker_id']
bikers_netw_info.index = new_indx
bikers_netw_info.drop('biker_id', axis=1, inplace=True)
display(bikers_netw_info)

,friends
biker_id,
DB97468391,BD46449342 DI73244116 EC26080662 BC22907620 FE...
DF37982273,BE91560444 DJ5798035 CA36380346 IJ9375619 DF34...
IC3183725,BE84954627 BJ50387873 BG52977611 EB85960823 EC...
BI72223848,ID361640 HC3814682 FF7944478 BH24049724 CF3059...
DE29017717,EC53303705 CB30310957 BI38389374 DJ28735761 HB...
...,...
CD21558820,GB9730283 DB56233255 BH27834076 CD22451089 DI5...
EA10747551,BF52872602 DA85585165 BF57664371 CH43735210 CB...
CI0076614,EA83058327 IB2658711 BC22305296 JF3357686 DC42...


In [ ]:
biker_friends = np.zeros((38209, 38209), dtype=np.int8)
for biker_id in tqdm(biker_ids):
    try:
        friends = bikers_netw_info.loc[biker_id]['friends'].split()
        for friend in friends:
            if friend in biker_ids:
                biker_friends[bikers_dict[biker_id]][bikers_dict[friend]] = 1
                biker_friends[bikers_dict[friend]][bikers_dict[biker_id]] = 1
    except:
        continue
        
biker_friends

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [ ]:
biker_network = pd.DataFrame(data=biker_friends, index=biker_ids, columns=biker_ids)
display(biker_network)

,DB97468391,DF37982273,IC3183725,BI72223848,DE29017717,GC7175141,CH52000443,DE73687777,CJ66052962,CG4876277,BF34483818,CG48135297,BD55996271,CE11726276,BG92345316,DH86316231,CA81587,BI4647001,BA13376584,CG86249984,CC76008938,BA91720544,DF93684261,ID9960040,DB84414062,IE133998,IF1286067,DD41263967,BD63661070,BJ11472832,BB86129084,DH04022334,CD96669325,CD70879614,BH48125486,DE81126833,GI7155152,DB04149470,EA03794851,BA00773879,...,CI90755396,DE08582003,HF1622875,CD8825309,IG4498963,CI16382340,HE551074,BB86306562,DE51630123,JJ874347,CH19488855,CJ82334896,BD11900770,BA13322316,EA21965074,GA2148753,JI5245482,EB96819363,DF66369790,BH21788513,DG64262281,CG30548270,BF8436714,DI83859262,DA21150625,CI43204471,DH16696763,BE40486303,CD21558820,EA10747551,CI0076614,CB87669797,FE820382,DF71403566,DI90944219,EA64394129,EE6388078,CF48659919,HC3239978,BC5781176
DB97468391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DF37982273,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
IC3183725,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BI72223848,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DE29017717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EA64394129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
EE6388078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CF48659919,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HC3239978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
biker_network.to_csv('./Required data/biker_network.csv')